Lecture: AI I - Advanced 

Previous:
[**Chapter 3.1: PyTorch Ligning**](../01_lightning.ipynb)

---

# Exercise 3.1: PyTorch Lightning

> Hint: When doing the exercises put your solution in the designated "Solution" section:
> ```python
> # Solution (put your code here)
> ```

## Task 1: Diabetes Regression with Multi-Layer Perceptron 

The diabetes dataset contains 442 samples with 10 baseline variables (age, sex, BMI, blood pressure, and 6 blood serum measurements). The target is a quantitative measure of disease progression one year after baseline.

**Tasks**
- Data Exploration & Understanding
- Data preparation
- Build Model a Multi-Layer Perceptron for regression but use PyTorch Lightning
- Train the model 
- Evaluate the model performance using appropriate regression metrics (e.g. MSE, MAE) and Tensorboard

In [1]:
# prerequisites (don't edit this block)
from sklearn.datasets import load_diabetes

diabetes = load_diabetes()
x = diabetes.data
y = diabetes.target

In [2]:
# Solution (put your code here)
import torch

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset, DataLoader

torch.manual_seed(42)  # set random seed for reproducibility

x_scaled = StandardScaler().fit_transform(x)
x_train, x_temp, y_train, y_temp = train_test_split(
    x_scaled, y, test_size=0.4, random_state=42
)
x_val, x_test, y_val, y_test = train_test_split(
    x_temp, y_temp, test_size=0.5, random_state=42
)

batch_size = 16
train_loader = DataLoader(TensorDataset(torch.tensor(x_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32).view(-1,1)), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(TensorDataset(torch.tensor(x_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.float32).view(-1,1)), batch_size=batch_size)
test_loader = DataLoader(TensorDataset(torch.tensor(x_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32).view(-1,1)), batch_size=batch_size)

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import lightning as L

class DiabetesClassifier(L.LightningModule):
    """
    Lightning module for Diabetes classification.
    All model logic in one organized place!
    """
    def __init__(self, input_dim=10, hidden_dim=64, output_dim=1, learning_rate=0.001, dropout=0.2):
        super().__init__()
        self.save_hyperparameters() # Save hyperparameters (automatically logged!)
        
        self.layer1 = nn.Linear(input_dim, hidden_dim)
        self.dropout = nn.Dropout(dropout)
        self.layer2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        """Forward pass - same as PyTorch"""
        x = F.relu(self.layer1(x))
        x = self.dropout(x)
        x = self.layer2(x)
        return x
    
    def _step(self, step_name, batch, batch_idx):
        """Common step for training, validation, and testing"""
        x, y = batch
        y_hat = self(x)
        #loss = F.cross_entropy(y_hat, y)
        loss = nn.MSELoss()

        preds = torch.argmax(y_hat, dim=1)
        acc = (preds == y).float().mean()
        
        # Log metrics (automatically sent to logger)
        self.log(f'{step_name}_loss', loss, prog_bar=True)
        self.log(f'{step_name}_acc', acc, prog_bar=True)
        return loss
    
    def training_step(self, batch, batch_idx):
        """
        Defines what happens in ONE training step.
        Lightning handles the loop, optimization, and device placement!
        """
        return self._step('train', batch, batch_idx) 
    
    def validation_step(self, batch, batch_idx):
        """What happens in ONE validation step"""
        self._step('val', batch, batch_idx) 

    def test_step(self, batch, batch_idx):
        """What happens in ONE test step"""
        self._step('test', batch, batch_idx)

    def configure_optimizers(self):
        """
        Define optimizer and (optionally) learning rate scheduler.
        Return optimizer or dict with optimizer and scheduler.
        """
        optimizer = torch.optim.Adam(self.parameters(), lr=self.hparams.learning_rate) 
        return {'optimizer': optimizer}

In [7]:
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint
from lightning.pytorch.loggers import TensorBoardLogger


model = DiabetesClassifier(hidden_dim=64, learning_rate=0.001, dropout=0.2)
callbacks = [
    # Early stopping: stop when val_loss doesn't improve for 15 epochs
    EarlyStopping(
        monitor='val_loss',
        patience=15,
        mode='min',
        verbose=True
    ),
    
    # Model checkpoint: save top 3 models based on val_loss
    ModelCheckpoint(
        monitor='val_loss',
        dirpath='./data/01_lightning/models/',
        filename='iris-{epoch:02d}-{val_loss:.4f}',
        save_top_k=3,
        mode='min'
    )
]

trainer = L.Trainer(
    max_epochs=100,
    accelerator='auto',  # Automatically use GPU if available
    devices='auto',
    callbacks=callbacks,
    logger=TensorBoardLogger('./data/01_lightning/logs', name='diabetes_classifier'),
    log_every_n_steps=5,
    enable_progress_bar=True
)
trainer.fit(model, train_loader, val_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.


┏━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name    ┃ Type    ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ layer1  │ Linear  │    704 │ train │     0 │
│ 1 │ dropout │ Dropout │      0 │ train │     0 │
│ 2 │ layer2  │ Linear  │     65 │ train │     0 │
└───┴─────────┴─────────┴────────┴───────┴───────┘

Trainable params: 769                                                                                              
Non-trainable params: 0                                                                                            
Total params: 769                                                                                                  
Total estimated model params size (MB): 0                                                                          
Modules in train mode: 3                                                                                           
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

Output()

ValueError: `self.log(val_loss, MSELoss())` was called, but `MSELoss` values cannot be logged

In [ ]:
# Test case (don't edit this block)
assert True

---

Lecture: AI I - Advanced 

Next: [**Chapter 4.1.1: Convolutional Neural Networks**](../../04_excursus/01_cv/01_cnn.ipynb)